In [ ]:
using ModelingToolkit 
using DomainSets
using MethodOfLines
using NonlinearSolve
using DifferentialEquations
include("multiharmonic_balance.jl");
using Random
using Base.Threads

# Plotting
using Plots

# FFTW 
using FFTW

In [ ]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
yleft = 0.0;
yright = 1.0;
Nt = 5
Nx = Ny = 40;
harmonics = 1; # number of harmonics
order = 2;
stepx = (xright-xleft)/Nx;
stepy = (yright - yleft)/Ny;
u0 = 0.005 * zeros((Nx+1) * (Ny+1) * harmonics * 2);
N = (Nx+1) * (Ny+1);

gamma = 0.0;
gamma3_final = 0.0;
g0::Float64 = 1; # m / s^2
height = 1; # m
harmonics = 3; # number of harmonics
A_forcing::Float64 = 1000;
lambda_forcing::Float64 = -40;
omega = 2.0;

#Define parameters and variables
@parameters x y t
@variables u_FD(..)

#Define differential operators:
const Dt = Differential(t);
const Dx = Differential(x);
const Dy = Differential(y)
const Dtt = Differential(t)^2;
const Dxx = Differential(x)^2;

In [ ]:
function make_WaveEquation_2D(u, g0, height, gamma, gamma3, forcing)
    pde::Symbolics.Num = Dt(Dt(u)) - 0.25*(Dx(Dx(u)) + Dy(Dy(u))) + gamma*Dt(u) + gamma3*Dt(u)*Dt(u)*Dt(u) - forcing;
    return pde
end;

In [ ]:
u0 = rand((Nx + 1) * (Ny + 1) * harmonics * 2)*.4;
solHB = nothing 
u_hist = Vector{Float64}[]  # store history for prediction
gamma3_hist = Float64[]


vars, var_exprs, (u_HB,) = create_ansatz((x, y), t, omega, harmonics);
    
forcing = A_forcing * exp(lambda_forcing*x^2) * sin(omega * t)

bc_conds = Dict(
    :A1 => [:(0.0), 0.0],
    :B1 => [:(0.0), 0.0],
    :A2 => [:(0.0), 0.0],
    :B2 => [:(0.0), 0.0],
    :A3 => [:(0.0), 0.0],
    :B3 => [:(0.0), 0.0],
)

gamma3 = 0.0
dgamma3 = 0.1
resid = nothing

while gamma3 < gamma3_final
    timed_out = false
    println("Beginning the symbolics manipulation")
    flush(stdout)
    
    if length(u_hist) >= 2
        u0 = u_hist[end] + (u_hist[end] - u_hist[end-1]) * 
             dgamma3 / (gamma3_hist[end] - gamma3_hist[end-1])
    elseif length(u_hist) == 1
        u0 = u_hist[end]
    else
        u0 = rand((Nx+1) * (Ny+1) * harmonics * 2)*.3
    end
    
    
    pde::Symbolics.Num = make_WaveEquation_2D(u_HB, g0, height, gamma, gamma3, forcing);
    

    expanded = expand_trig_jl(pde, t, omega)
    eqns = make_equations(expanded, harmonics, omega, t)
    sym_eqs = map(transform_sym(Nx, Ny) ∘ Meta.parse ∘ string, eqns)
    
    resid = create_residual_function(sym_eqs, vars, Nx, Ny, bc_conds)
    
    residual! = eval(resid)
    
    # DiffMat, LaplCoeff = create_jac_blocks(eqns, var_exprs, harmonics)
    
    # jacobian = create_jacobian_function(Nx + 1, DiffMat, LaplCoeff, harmonics)
    
    func = NonlinearFunction(residual!)
    prob = NonlinearProblem(func, u0, [stepx, stepy])
    
    solHB = solve(prob, RobustMultiNewton(), reltol=1e-5, abstol=1e-5, maxiters=1000)

    u0 = solHB.u
    println(round(gamma3, sigdigits=6), " ", solHB.retcode)
    if solHB.retcode == ReturnCode.Success
        println("Succeeded!")
        flush(stdout)
        push!(u_hist, copy(solHB.u))
        push!(gamma3_hist, gamma3)
        dgamma3 = min(dgamma3 * 1.2, 0.02)
        
        gamma3 += dgamma3

        if gamma3 > gamma3_final
            gamma3 = gamma3_final
            break
        end
        
    else
        println("Failed! Going back to the previous gamma and reducing the step!")
        flush(stdout)
        gamma3 -= dgamma3
        dgamma3 /= 10
        gamma3 += dgamma3
        dgamma3 < 1e-6 && break
    end


    
end

pde::Symbolics.Num = make_WaveEquation_2D(u_HB, g0, height, gamma, gamma3, forcing);
    
expanded = expand_trig_jl(pde, t, omega)
eqns = make_equations(expanded, harmonics, omega, t)
sym_eqs = map(transform_sym(Nx, Ny) ∘ Meta.parse ∘ string, eqns)

resid = create_residual_function(sym_eqs, vars, Nx, Ny, bc_conds);

residual! = eval(resid)

# DiffMat, LaplCoeff = create_jac_blocks(eqns, var_exprs, harmonics)

# jacobian = create_jacobian_function(Nx + 1, DiffMat, LaplCoeff, harmonics)

func = NonlinearFunction(residual!)
prob = NonlinearProblem(func, u0, [stepx, stepy])

println("Starting now the FINAL iteration")
flush(stdout)

@time solHB = solve(prob, RobustMultiNewton(), reltol=1e-5, abstol=1e-5, maxiters=1000)

print(solHB.retcode)

N = (Nx+1)*(Ny+1)
coefficientsHB = [reshape(solHB.u[(k-1)*N+1:k*N], Nx+1, Ny+1) for k in 1:(2*harmonics)];

In [ ]:
maximum(coefficientsHB[1])

In [ ]:
maxlim  = maximum(coefficientsHB[1])*1.2
x = range(0, 1, length=Nx+1)
y = range(0, 1, length=Ny+1)

T = 2π / omega
n_frames = 500
anim = @animate for t in range(0, T, length=n_frames)
    u_new = coefficientsHB[1] * 0.0
    j = 1
    for i in 1:(2*harmonics)
        if isodd(i)
            u_new .+= coefficientsHB[i] .* sin(j * omega * t)
        else
            u_new .+= coefficientsHB[i] .* cos(j * omega * t)
            j += 1
        end
    end
    
    surface(x, y, u_new', 
        zlims=(-maxlim, maxlim), 
        clims=(-maxlim, maxlim),
        xlabel="x", ylabel="y", zlabel="u",
    title = "H=$harmonics, ω=$(round(omega, digits=1)), " *
            "γ=$gamma, γ₃=$(round(gamma3, sigdigits=2))\n" *
            "Nx=$Nx, Ny=$Ny, t=$(round(t, digits=2)))",
        camera=(30, 30)  # viewing angle
    )
end
gif(anim, "harmonicBalancePlots/WaveEq_gino_$(harmonics)H.gif", fps=60)